# Basic regression: Predict Corrosion as Epit

*This notebook is based on the [tutorial notebook](https://www.tensorflow.org/tutorials/keras/regression) provided by TensorFlow.*

This Notebook takes the data that has been cleaned and imputed in book 2. If I have time I would produce another set of the data with the imputation according to each metal plot as file impute Nan Intuitively.

This example uses the `tf.keras` API, see [this guide](https://www.tensorflow.org/guide/keras) for details.

## Setup

In [10]:
import datetime, time, os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,  mean_squared_error, mean_absolute_error

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [11]:
import tensorflow as tf
import pathlib
import shutil
import tempfile

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.8.0


In [12]:
!pip install -q git+https://github.com/tensorflow/docs
    
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

### Setup for TensorBoard

We will use the TensorBoard to visualize some results. You can find more information and the board itself at the end of this notebook, but we will define the path were the information should be stored directly here at the beginning. 

In [13]:
# With this command you can clear any logs from previous runs
# If you want to compare different runs you can skip this cell 
!rm -rf my_logs/

In [14]:
# Define path for new directory 
root_logdir = os.path.join(os.curdir, "my_logs")
# Define function for creating a new folder for each run
def get_run_logdir():
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [15]:
# Create function for using callbacks; "name" should be the name of the model you use
def get_callbacks(name):
    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

In [18]:
# Define seeds etc.
RSEED = 42
tf.keras.backend.set_floatx('float64')
plt.rcParams['figure.figsize'] = (15, 10)

## The Alloys Data
The dataset is available from book 2_Metal Features, already cleaned.

In [16]:
Alloys= pd.read_csv("../data/Alloy_Inter.csv")

In [17]:
Alloys.head(1)

,Cl,Test_Temp,pH,M_Al,M_Fe,M_HEA,M_NiCrMo,M_Other,Fe,Cr,...,Ta,Re,Ce,Ti,Co,B,Mg,Y,Gd,Epit
0,0.005133,50.0,7.8,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.0


### Split the data into train and test

Now we'll split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our models.

In [ ]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(Alloys, target, random_state=RSEED)

In [ ]:
Alloys_train = Alloys.sample(frac=0.8, random_state=42)
Alloys_test = Alloys.drop(Alloys_train.index)

In [ ]:
Alloys.columns

Overall statistics, note how each feature covers a very different range:

In [ ]:
Alloys_test.describe().transpose()

### Split features from labels

Before we can start with the modelling process we need to separate our label from the dataset. This label is the value that we will train the model to predict.

In [ ]:
X_train = Alloys_train.copy()
X_test =  Alloys_test.copy()

y_train = X_train.pop('Epit')
y_test = X_test.pop('Epit')

## Normalization

In the table of statistics it's easy to see how different the ranges of each feature are.

In [ ]:
X_train.describe().transpose()[['mean', 'std']]
X_train.describe()

It is good practice to normalize features that use different scales and ranges. In our case we have the metal features, Cl, pH, Test_temp that present no gausian shape, so it is necesary to normalise them rather than standarise them.

One reason this is important is because the features are multiplied by the model weights. So the scale of the outputs and the scale of the gradients are affected by the scale of the inputs. 

Although a model *might* converge without feature normalization, normalization makes training much more stable. 

### The Normalization layer
The `preprocessing.Normalization` layer is a clean and simple way to build that preprocessing into your model.

The first step is to create the layer:

In [ ]:
normalizer = preprocessing.Normalization()

Then `.adapt()` it to the data:

In [ ]:
normalizer.adapt(np.array(X_train))

This calculates the mean and variance, and stores them in the layer. 

In [ ]:
normalizer.mean.numpy()

When the layer is called it returns the input data, with each feature independently normalized. We can have a look at the first training instance and compare the original and normalized features:

In [ ]:
first = np.array(X_train[:1])

with np.printoptions(precision=2, suppress=True):
    print('First example:', first)
    print()
    print('Normalized:', normalizer(first).numpy())

## A DNN regression

The previous section implemented linear models for single and multiple inputs.

This section implements single-input and multiple-input DNN models. The code is basically the same except the model is expanded to include some "hidden"  non-linear layers. The name "hidden" here just means not directly connected to the inputs or outputs.

These models will contain a few more layers than the linear model:

1. The normalization layer.
2. Two hidden, nonlinear, `Dense` layers using the `relu` nonlinearity.
3. A linear single-output layer.

Both will use the same training procedure so we'll include the `compile` method in the newly defined `build_and_compile_model` function below.

In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(164, activation='relu', input_dim=28),
        layers.Dense(164, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mae',
                  metrics='mse',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

### One variable

Let's start with a DNN model for a single input: "Fe".

1. First create the Fe `Normalization` layer:

In [ ]:
Fe = np.array(X_train['Fe'])

Fe_normalizer = preprocessing.Normalization(input_shape = [1,], axis = None)
Fe_normalizer.adapt(Fe)

2. Then we'll build the sequential model:

In [ ]:
Fe_model = tf.keras.Sequential([
    Fe_normalizer,
    layers.Dense(units=1)])

# We can print a summary of the model architecture with the following line:
#Fe_model.summary()

In [ ]:
dnn_Fe_model = build_and_compile_model(Fe_normalizer)

This model has quite a few more trainable parameters than the linear models.

In [ ]:
dnn_Fe_model.summary()

Let's train the model:

In [ ]:
%%time
history = dnn_Fe_model.fit(
    X_train['Fe'], y_train,
    validation_split=0.2,
    verbose=0, epochs=100,
    callbacks=get_callbacks("dnn_Fe"))

This model does slightly better than the linear-Fe model.

In [ ]:
plot_loss(history)

If you plot the predictions as a function of `Fe`, you'll see how this model takes advantage of the nonlinearity provided by the hidden layers:

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = dnn_Fe_model.predict(x)

In [ ]:
plot_Fe(x, y)

We'll also collect the results on the test set, for later:

In [ ]:
test_results['dnn_Fe_model'] = dnn_Fe_model.evaluate(
    X_train['Fe'], y_train,
    verbose=0)

### Full model

If you repeat this process using all the inputs it slightly improves the performance on the validation dataset.

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=0, epochs=100,
    callbacks=get_callbacks("dnn_model"))

In [ ]:
plot_loss(history)

Collect the results on the test set:

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(X_test, y_test, verbose=0)

## Performance

Now that all the models are trained let's check the test-set performance and compare how they performed:

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [Epit]', 'Root Mean squared error [Epit]', 'R2[Epit]', 'Ratio[Epit]']).T

These results match the validation error seen during training.

### Make predictions

Finally, predict have a look at the errors made by the model when making predictions on the test set:

In [ ]:
y_pred = dnn_model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred)
plt.xlabel('True Values [Epit]')
plt.ylabel('Predictions [Epit]')
lims = [-1000,  1800]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

It looks like the model predicts reasonably well. 

Now take a look at the error distribution:

In [ ]:
error = y_pred - y_test
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Epit]')
_ = plt.ylabel('Count')

If you're happy with the model you can save it for later use:

In [ ]:
dnn_model.save('dnn_model')

If you reload the model, it will give you identical outputs:

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    X_test, y_test, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [Epit]', 'Root Mean squared error [Epit]', 'R2[Epit]', 'Ratio[Epit]']).T

## Conclusion

This notebook introduced a few techniques to handle a regression problem. Here are a few more tips that may help:

* [Mean Squared Error (MSE)](https://www.tensorflow.org/api_docs/python/tf/losses/MeanSquaredError) and [Mean Absolute Error (MAE)](https://www.tensorflow.org/api_docs/python/tf/losses/MeanAbsoluteError) are common loss functions used for regression problems. Mean Absolute Error is less sensitive to outliers. Different loss functions are used for classification problems.
* Similarly, evaluation metrics used for regression differ from classification.
* When numeric input data features have values with different ranges, each feature should be scaled independently to the same range.
* Overfitting is a common problem for DNN models, it wasn't a problem for this tutorial. See the [overfit and underfit](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) tutorial for more help with this.


---
## TensorBoard

In machine learning, to improve something you often need to be able to measure it. TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

TensorBoard can be used directly within a notebook in Colab and Jupyter. This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

You can find a nice introductory notebook [here](https://www.tensorflow.org/tensorboard/get_started).

When you are running this NB in a browster and the tensorboard cannot be displayed: If you are using Safari, try to switch to Google Chrome and run it again.

### Example using TensorBoard

When training with Keras's `.fit()`, adding the `tf.keras.callbacks.TensorBoard` callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with histogram_freq=1 (this is turned off by default).

We've saved the logs in a timestamped subdirectory to allow easy selection of different training runs.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

You can start TensorBoard within the notebook using magics. At the end of the command you need to specify the path where the log files are saved.

In [ ]:
%tensorboard --logdir=./my_logs